In [ ]:
# Work_03
## Writing functions to detect contours
## Writing functions to calculate geometric attributes and spectral features
## Summarizing calculation of geometric attributes and spectral features into one function 

In [5]:
# loading libraries
import torch
torch.cuda.empty_cache() 
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:100000"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
os.getcwd()
import glob
import shapely
from shapely.geometry import Polygon
import shapely.plotting
import pandas as pd
from PIL import Image as ima
import warnings
from pathlib import Path

In [8]:
# loading written functions
from mtp_function_yl import *

In [ ]:
# write function to find and plot filtered contours (find-filter-plot-contour)
def ffpcontour(image, mask, i):
    image_masked = cv2.bitwise_and(image,image,mask = mask[i])
    assert image is not None, "image file could not be read, check with os.path.exists()"
    assert mask is not None, "mask file could not be read, check with os.path.exists()"
    ret, thresh = cv2.threshold((mask[i]*255), 127, 255, 0)
    
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 1:
        contour_f = contours
    else:
        contour_f = []
        for i in range(0, len(contours)):
            # print(i, "len", len(contours[i]))
            if len(contours[i]) > 80:
                contour_f.append(contours[i])
    img_con = cv2.drawContours(image_masked, contour_f, -1, (0,255,0), 3) 
    plt.figure(figsize = (15,15))
    plt.imshow(img_con)
    plt.axis('on')
    plt.show
    return contour_f


# update function
## Filtering the masks which are too small (<=5) for contour feature/properties calculation
# Function to find and plot filtered contours (find-filter-plot-contour)
# Run for each mask of each image
def ffpcontour_noplot(image, mask, i):
    assert image is not None, "image file could not be read, check with os.path.exists()"
    assert mask is not None, "mask file could not be read, check with os.path.exists()"
    ret, thresh = cv2.threshold((mask[i]*255), 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ll = [] # length list
    for i in range(0, len(contours)):
          ll.append(len(contours[i]))
    maxl = max(ll)
    maxindex = ll.index(maxl)
    if (len(contours) == 1) and (maxl >= 6):
        contour_f = contours
    elif (maxl>=80) :
        contour_f = []
        for i in range(0, len(contours)):
        # print(i, "len", len(contours[i]))
            if (len(contours[i]) >= 80):
                contour_f.append(contours[i])
            else:
                contour_f = contour_f
    elif (maxl >= 6):
        contour_f = []
        contour_f.append(contours[maxindex])
    else: 
        contour_f = []
    return contour_f


In [ ]:
# write a function to calculate geometric attributes

def cgr(contour):
    assert contour is not None, "image file could not be read, check with os.path.exists()"
    c = contour
    isconvex = cv2.isContourConvex(c) # Checking convexity
    (x,y), (w,h), ar = cv2.minAreaRect(c) # Rotated rectangle with minimum area
    ar_r = ar # Angle of rotation in rotated rectangle
    M = cv2.moments(c) # Moments
    area = cv2.contourArea(c) # Area 
    if (M['m00'] != 0):
        cx = int(M['m10']/M['m00']) # Centroid
        cy = int(M['m01']/M['m00'])
    else:
        cx = x
        cy = y
    xs,ys,ws,hs = cv2.boundingRect(c) # Straight bounding rectangle
    aspect_ratio_wh_s = float(ws)/hs # Aspect ratio
    extent_s = float(area)/(ws*hs) # Extent
    hull = cv2.convexHull(c) # Solidity
    hull_area = cv2.contourArea(hull)
    if (hull_area != 0):
        solidity = float(area)/hull_area
    else:
        solidity = 0
    aspect_ratio_wh = float(w)/h  # Aspect ratio
    extent = float(area)/(w*h) # Extent
    (xe,ye),(MA,ma),ae = cv2.fitEllipse(c)
    ar_e = ae # Angle of rotated ellipse, orientation
    ed = np.sqrt(4*area/np.pi) # Equivalent Diameter
    ratio_ell = float(ma)/MA
    perimeter = cv2.arcLength(c, True) # Arclength
    p_centroid = np.array([float(cx), float(cy)])
    p_masscenter = np.array([float(x), float(y)])
    # for the following two variable, positive (+1) = inside, negative (-1) = outside, or zero (0) = on an edge
    is_cen_inside = cv2.pointPolygonTest(c, p_centroid, False) # Checking if centroid is inside
    is_mce_inside = cv2.pointPolygonTest(c, p_masscenter, False) # Checking if mass center is inside
    leftmost = tuple(c[c[:,:,0].argmin()][0])
    rightmost = tuple(c[c[:,:,0].argmax()][0])
    topmost = tuple(c[c[:,:,1].argmin()][0])
    bottommost = tuple(c[c[:,:,1].argmax()][0])
    leftm = leftmost[0]
    rightm = rightmost[0]
    topm = topmost[1]
    bottomm = bottommost[1]
    return {
        'isconvex': isconvex,
        'area': area,
        'aspect_ratio_wh_s': aspect_ratio_wh_s,
        'extent_s': extent_s,
        'solidity': solidity,
        'aspect_ratio_wh': aspect_ratio_wh,
        'extent': extent,
        'orien_rre': ar_r,
        'orien_ell': ar_e,
        'ed': ed,
        'ratio_ell': ratio_ell,
        'perimeter': perimeter,
        'is_cen_inside': is_cen_inside,
        'is_mce_inside': is_mce_inside,
        'leftm': leftm,
        'rightm': rightm,
        'topm': topm,
        'bottomm': bottomm
    }

# update the function
def csga(contours):
    assert contours is not None, "image file could not be read, check with os.path.exists()"
    if len(contours) == 1:
        ga = cgr(contours[0])
    else:
        gal = []
        for i in range(0, (len(contours)-1)):
            gal.append(cgr(contours[i]))
        iscon = []
        al = []
        asps = []
        exts = []
        sol = []
        asp = []
        ext = []
        anr = []
        ane = []
        ed = []
        rate = []
        per = []
        iscen = []
        ismce = []
        lm = []
        rm = []
        tm = []
        bm = []
        for i in range(0, len(gal)):
            iscon.append(gal[0]['isconvex'])
            al.append(gal[0]['area'])
            asps.append(gal[0]['aspect_ratio_wh_s'])
            exts.append(gal[0]['extent_s'])
            sol.append(gal[0]['solidity'])
            asp.append(gal[0]['aspect_ratio_wh'])
            ext.append(gal[0]['extent'])
            anr.append(gal[0]['orien_rre'])
            ane.append(gal[0]['orien_ell'])
            ed.append(gal[0]['ed'])
            rate.append(gal[0]['ratio_ell'])
            per.append(gal[0]['perimeter'])
            iscen.append(gal[0]['is_cen_inside'])
            ismce.append(gal[0]['is_mce_inside'])
            lm.append(gal[0]['leftm'])
            rm.append(gal[0]['rightm'])
            tm.append(gal[0]['topm'])
            bm.append(gal[0]['bottomm'])
        isconvex = np.all(iscon)
        area = np.mean(al, axis = 0)
        aspect_ratio_wh_s = np.mean(asps, axis = 0)
        extent_s = np.mean(exts, axis = 0)
        solidity = np.mean(sol, axis = 0)
        aspect_ratio_wh = np.mean(asp, axis = 0)
        extent = np.mean(ext, axis = 0)
        ar_r = np.mean(anr, axis = 0)
        ar_e = np.mean(ane, axis = 0)
        ed = np.mean(ed, axis = 0)
        ratio_ell = np.mean(rate, axis = 0)
        perimeter = np.mean(per, axis = 0)
        is_cen_inside = np.mean(iscen, axis = 0)
        is_mce_inside = np.mean(ismce, axis = 0)
        leftm = np.min(lm, axis = 0)
        rightm = np.max(rm, axis = 0)
        topm = np.min(tm, axis = 0)
        bottomm = np.max(bm, axis = 0)
        ga = {
            'isconvex': isconvex,
            'area': area,
            'aspect_ratio_wh_s': aspect_ratio_wh_s,
            'extent_s': extent_s,
            'solidity': solidity,
            'aspect_ratio_wh': aspect_ratio_wh,
            'extent': extent,
            'orien_rre': ar_r,
            'orien_ell': ar_e,
            'ed': ed,
            'ratio_ell': ratio_ell,
            'perimeter': perimeter,
            'is_cen_inside': is_cen_inside,
            'is_mce_inside': is_mce_inside,
            'leftm': leftm,
            'rightm': rightm,
            'topm': topm,
            'bottomm': bottomm
        }
    return ga



In [ ]:
# # checking the contour, for example

# # which image
# j = 1
# mb = mask3[j]
# ima = gsv_image3[j]

# # which mask
# # i = i+1
# i = 4 
# print(i)
# maskedimg = cv2.bitwise_and(ima,ima,mask = mb[i])
# # create a mask
# hist_full = cv2.calcHist([ima],[0],None,[256],[0,256])
# hist_mask = cv2.calcHist([ima],[0],mb[i],[256],[0,256])
# plt.figure(figsize = (10,10))
# plt.subplot(221), plt.imshow(ima)
# plt.subplot(222), plt.imshow(mb[i])
# plt.subplot(223), plt.imshow(maskedimg)
# plt.subplot(224), plt.plot(hist_full), plt.plot(hist_mask)
# plt.xlim([0,256])
# plt.show()

In [ ]:
# write a function to calculate color distance between RGB points to gray line
def lineseg_dist(p, a, b):
    # normalized tangent vector
    d = np.divide(b - a, np.linalg.norm(b - a))
    # signed parallel distance components
    s = np.dot(a - p, d)
    t = np.dot(p - b, d)
    # clamped parallel distance
    h = np.maximum.reduce([s, t, 0])
    # perpendicular distance component
    c = np.cross(p - a, d)
    return np.hypot(h, np.linalg.norm(c))

def coldistance(mig, mib, mir):
    cdmean = []
    cdstd = []
    for i in range(0,len(mib)):
        cg = mig[i]
        cr = mir[i]
        cb = mib[i]
        diss = []
        t2 = np.array([0,0,0])
        t3 = np.array([255,255,255])
        for j in range(0, len(cg)):
            diss.append(lineseg_dist(np.array([cg[j], cr[j], cb[j]]), t2, t3))
        cdmean.append(np.mean(diss))
        cdstd.append(np.std(diss))
    return cdmean, cdstd


In [ ]:
# summarize calculation of geometric attributes and spectral features into one function
def feature_summary(image, mf):
    # Generate a data frame for masks and attributes
    df = pd.DataFrame()
    df['mask'] = range(1, (len(mf)+1))
    df = df.assign(gmedian = None, rmedian = None, bmedian = None,
                   gmean = None, rmean = None, bmean = None,
                   gstd = None, rstd = None, bstd = None,
                   gq25 = None, gq75 = None, rq25 = None,
                   rq75 = None, bq25 = None, bq75 = None,
                   cdmean = None, cdstd = None,
                   isconvex = None, area = None, aspect_ratio_wh_s = None,
                   extent_s = None, solidity = None, aspect_ratio_wh = None,
                   extent = None, orien_rre = None, orien_ell = None,
                   ed = None, ratio_ell = None,
                   perimeter = None, is_cen_inside = None, is_mce_inside = None,
                   leftm = None, rightm = None, topm = None, 
                   bottomm = None)
    mm = [] # masked image
    for i in range(0, len(mf)):
        mm.append(cv2.bitwise_and(image, image, mask = mf[i]))
    mib = []
    mig = []
    mir = []
    for i in range(0, len(mm)):
        mib.append((mm[i][:,:,0])[np.where(((mm[i][:,:,0]) != 0) | ((mm[i][:,:,1]) != 0) | ((mm[i][:,:,2]) != 0))])
        mig.append((mm[i][:,:,1])[np.where(((mm[i][:,:,0]) != 0) | ((mm[i][:,:,1]) != 0) | ((mm[i][:,:,2]) != 0))])
        mir.append((mm[i][:,:,2])[np.where(((mm[i][:,:,0]) != 0) | ((mm[i][:,:,1]) != 0) | ((mm[i][:,:,2]) != 0))])
    cm, cs = coldistance(mig, mib, mir)
    for i in range(0, len(mm)):
        df.at[i, 'bmean'] = np.mean(mib[i], axis = 0)
        df.at[i, 'gmean'] = np.mean(mig[i], axis = 0)
        df.at[i, 'rmean'] = np.mean(mir[i], axis = 0)
        df.at[i, 'bmedian'] = np.median(mib[i], axis = 0)
        df.at[i, 'gmedian'] = np.median(mig[i], axis = 0)
        df.at[i, 'rmedian'] = np.median(mir[i], axis = 0)
        df.at[i, 'bstd'] = np.std(mib[i], axis = 0)
        df.at[i, 'gstd'] = np.std(mig[i], axis = 0)
        df.at[i, 'rstd'] = np.std(mir[i], axis = 0)
        df.at[i, 'bq25'] = np.quantile(mib[i], 0.25, axis = 0)
        df.at[i, 'bq75'] = np.quantile(mib[i], 0.75, axis = 0)
        df.at[i, 'gq25'] = np.quantile(mig[i], 0.25, axis = 0)
        df.at[i, 'gq75'] = np.quantile(mig[i], 0.75, axis = 0)
        df.at[i, 'rq25'] = np.quantile(mir[i], 0.25, axis = 0)
        df.at[i, 'rq75'] = np.quantile(mir[i], 0.75, axis = 0)
        df.at[i, 'cdmean'] = cm[i]
        df.at[i, 'cdstd'] = cs[i]
    for i in range(0, len(mf)):
        con = ffpcontour_noplot(image, mf, i)
        if len(con) != 0 :
            df.at[i, 'isconvex'] = csga(con)['isconvex']
            df.at[i, 'area'] = csga(con)['area']
            df.at[i, 'aspect_ratio_wh_s'] = csga(con)['aspect_ratio_wh_s']
            df.at[i, 'extent_s'] = csga(con)['extent_s']
            df.at[i, 'solidity'] = csga(con)['solidity']
            df.at[i, 'aspect_ratio_wh'] = csga(con)['aspect_ratio_wh']
            df.at[i, 'extent'] = csga(con)['extent']
            df.at[i, 'ed'] = csga(con)['ed']
            df.at[i, 'orien_rre'] = csga(con)['orien_rre']
            df.at[i, 'orien_ell'] = csga(con)['orien_ell']
            df.at[i, 'ratio_ell'] = csga(con)['ratio_ell']
            df.at[i, 'perimeter'] = csga(con)['perimeter']
            df.at[i, 'is_cen_inside'] = csga(con)['is_cen_inside']
            df.at[i, 'is_mce_inside'] = csga(con)['is_mce_inside']
            df.at[i, 'leftm'] = csga(con)['leftm']
            df.at[i, 'rightm'] = csga(con)['rightm']
            df.at[i, 'topm'] = csga(con)['topm']
            df.at[i, 'bottomm'] = csga(con)['bottomm']
        else :
            df.at[i, 'isconvex'] = np.nan
            df.at[i, 'area'] = np.nan
            df.at[i, 'aspect_ratio_wh_s'] = np.nan
            df.at[i, 'extent_s'] = np.nan
            df.at[i, 'solidity'] = np.nan
            df.at[i, 'aspect_ratio_wh'] = np.nan
            df.at[i, 'extent'] = np.nan
            df.at[i, 'ed'] = np.nan
            df.at[i, 'orien_rre'] = np.nan
            df.at[i, 'orien_ell'] = np.nan
            df.at[i, 'ratio_ell'] = np.nan
            df.at[i, 'perimeter'] = np.nan
            df.at[i, 'is_cen_inside'] = np.nan
            df.at[i, 'is_mce_inside'] = np.nan
            df.at[i, 'leftm'] = np.nan
            df.at[i, 'rightm'] = np.nan
            df.at[i, 'topm'] = np.nan
            df.at[i, 'bottomm'] = np.nan
    # Remove the rows with na
    df = df.dropna()
    # Convert true/false to 1 and 0
    df = df.replace({True: 1, False: 0})
    return df
